In [1]:
import joblib
import pandas as pd
from nba_api.stats.endpoints import LeagueGameLog
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# داده‌های بازی‌ها را دریافت می‌کنیم
gamefinder = LeagueGameLog(season='2023-24', season_type_all_star='Regular Season')
games = gamefinder.get_data_frames()[0]
games = games[games['TEAM_ABBREVIATION'] != '']

# ساخت دیتافریم برای آموزش مدل
games['WIN'] = games['WL'] == 'W'
df = games[['GAME_ID', 'TEAM_NAME', 'PTS', 'WIN']]
df = df.merge(df, on='GAME_ID')
df = df[df['TEAM_NAME_x'] != df['TEAM_NAME_y']]

df['TEAM_1'] = df['TEAM_NAME_x']
df['TEAM_2'] = df['TEAM_NAME_y']
df['PTS_1'] = df['PTS_x']
df['PTS_2'] = df['PTS_y']
df['TARGET'] = df['PTS_1'] > df['PTS_2']

features = df[['TEAM_1', 'TEAM_2', 'PTS_1', 'PTS_2']]
target = df['TARGET']

# کدگذاری نام تیم‌ها
le_team = LabelEncoder()
features['TEAM_1'] = le_team.fit_transform(features['TEAM_1'])
features['TEAM_2'] = le_team.transform(features['TEAM_2'])

# تقسیم داده‌ها
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# مدل XGBoost و جستجوی تصادفی
param_dist = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'reg_alpha': [0.1, 0.5, 1.0],
    'reg_lambda': [0.1, 0.5, 1.0],
}

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
random_search = RandomizedSearchCV(xgb, param_distributions=param_dist, n_iter=10, cv=3, verbose=1)
random_search.fit(X_train, y_train)

# ذخیره مدل و کدگذار
model = random_search.best_estimator_
joblib.dump(model, 'nba_winner_model.pkl')
joblib.dump(le_team, 'team_encoder.pkl')

/tmp/ipykernel_133548/395474037.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['TEAM_1'] = le_team.fit_transform(features['TEAM_1'])
/tmp/ipykernel_133548/395474037.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['TEAM_2'] = le_team.transform(features['TEAM_2'])
/home/amirhosein/Desktop/project/NBA_model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [19:15:55] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

 

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/home/amirhosein/Desktop/project/NBA_model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [19:15:55] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/amirhosein/Desktop/project/NBA_model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [19:15:55] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/amirhosein/Desktop/project/NBA_model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [19:15:56] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/amirhosein/Desktop/project/NBA_model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [19:15:56] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not

['team_encoder.pkl']